In [2]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [7]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\DADOS\\415 - Repositório de Dados\\Repositório Local\\PEP -GF")

In [8]:
df = pd.read_csv("ingresso_GF_082023.csv", sep=";", decimal=",",thousands=".")

In [21]:
df=df[df['Ident. GDF']=='Gov. Federal']

In [9]:
df=df[['Ano',  "Tipo de Ingresso'", 'Regime Jurídico', 'Sexo',
       'Grupo do Cargo', 'Quantidade de Ingressos']]
df['Sexo']= np.where(df['Sexo']=='F',"Feminino",df['Sexo'])

In [10]:
df['Sexo']= np.where(df['Sexo']=='M',"Masculino",df['Sexo'])
df

,Ano,Tipo de Ingresso',Regime Jurídico,Sexo,Grupo do Cargo,Quantidade de Ingressos
0,2023,Processo Seletivo,Contrato Temporário,Feminino,Aposentado Contrato Temporario Inss,53
1,2023,Processo Seletivo,Contrato Temporário,Feminino,Car.Mag.Ensino Basico Tec.Tecn.Mp/431/08,15
2,2023,Processo Seletivo,Contrato Temporário,Feminino,Cargo Tempor Tec Especializ - Lei 8745,1
3,2023,Processo Seletivo,Contrato Temporário,Feminino,Carreira De Magisterio Superior,2746
4,2023,Processo Seletivo,Contrato Temporário,Feminino,Carreira Magist Ens Bas Tec Tecnol,1626
...,...,...,...,...,...,...
1943,2008,Concurso,Estatutário,Masculino,Plano Especial De Cargos Da Cultura,16
1944,2008,Concurso,Estatutário,Masculino,Plano Geral De Cargos Do Poder Executivo,102
1945,2008,Concurso,Estatutário,Masculino,Plano Geral De Cargos Poder Executivo-Ni,99
1946,2008,Concurso,Estatutário,Masculino,Receita Federal Brasil,2


In [11]:
df = df.rename(columns={'Ano':'ano',"Tipo de Ingresso'":'tipo_ingresso','Regime Jurídico':'regime_juridico','Sexo':'genero','Grupo do Cargo':'grupo_cargo','Quantidade de Ingressos':'quantidade_ingressos'})

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1929 entries, 0 to 2056
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ano                   1929 non-null   int64 
 1   tipo_ingresso         1929 non-null   object
 2   regime_juridico       1929 non-null   object
 3   genero                1929 non-null   object
 4   grupo_cargo           1929 non-null   object
 5   quantidade_ingressos  1929 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 105.5+ KB


In [43]:
df['regime_juridico'].unique()

array(['Contrato Temporário', 'Estatutário', 'Médico Residente',
       'Programa Mais Médicos', 'Residência Multi Profissional',
       'Regime Militar'], dtype=object)

In [12]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('tipo_ingresso','STRING',description='forma de ingresso no serviço público'),
 bigquery.SchemaField('regime_juridico','STRING',description='regime jurídico'),
 bigquery.SchemaField('genero','STRING',description='Qual gênero do ingressante'),
 bigquery.SchemaField('grupo_cargo','STRING',description='a qual grupo de cargo pertence'),
 bigquery.SchemaField('quantidade_ingressos','INTEGER',description='Quantidade de ingressos')
 ]

In [13]:
client = bigquery.Client()

In [14]:
dataset_ref = client.dataset('atracao_pre_selecao')
table_ref = dataset_ref.table('PEP_GF_ingressos')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=1b8f1784-3297-46cf-a2fb-240dedd52d1d>